# Basic tutorial: Question answer
#### Author: Matteo Caorsi

This short tutorial provides you with the basic functioning of *giotto-deep* API.

The example described in this tutorial is the one of question answer.

The main steps of the tutorial are the following:
 1. creation of a dataset
 2. creation of a model
 3. define metrics and losses
 4. train the model
 5. extract some features of the network

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np

import torch
from torch import nn

from gdeep.models import FFNet

from gdeep.visualisation import  persistence_diagrams_of_activations

from torch.utils.tensorboard import SummaryWriter
from gdeep.data import TorchDataLoader
from gdeep.pipeline import Pipeline

from gtda.diagrams import BettiCurve

from gtda.plotting import plot_betti_surfaces

No TPUs...


# Initialize the tensorboard writer

In order to analyse the reuslts of your models, you need to start tensorboard.
On the terminal, move inside the `/example` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.

In [2]:
writer = SummaryWriter()

# Create your dataset

In [3]:
from torch.utils.data.sampler import SubsetRandomSampler

# the only part of the training set we are interested in
train_indices = list(range(32*10))

dl = TorchDataLoader(name="SQuAD2", convert_to_map_dataset=True)
dl_tr_str, dl_ts_str = dl.build_dataloaders(sampler=SubsetRandomSampler(train_indices), batch_size=1)


The dataset contains a context and a question whose answer can be found within that context. The correct answer as well as the starting characters are also provided.

In [4]:
datum = next(iter(dl_tr_str))
datum

[('At the 57th Annual Grammy Awards in February 2015, Beyoncé was nominated for six awards, ultimately winning three: Best R&B Performance and Best R&B Song for "Drunk in Love", and Best Surround Sound Album for Beyoncé. She was nominated for Album of the Year but the award was won by Beck for his Morning Phase album. In August, the cover of the September issue of Vogue magazine was unveiled online, Beyoncé as the cover star, becoming the first African-American artist and third African-American woman in general to cover the September issue. She headlined the 2015 Made in America festival in early September and also the Global Citizen Festival later that month. Beyoncé made an uncredited featured appearance on the track "Hymn for the Weekend" by British rock band Coldplay, on their seventh studio album A Head Full of Dreams (2015), which saw release in December. On January 7, 2016, Pepsi announced Beyoncé would perform alongside Coldplay at Super Bowl 50 in February. Knowles has previou

In [5]:
datum = next(iter(dl_ts_str))
datum

[('The customary law of Normandy was developed between the 10th and 13th centuries and survives today through the legal systems of Jersey and Guernsey in the Channel Islands. Norman customary law was transcribed in two customaries in Latin by two judges for use by them and their colleagues: These are the Très ancien coutumier (Very ancient customary), authored between 1200 and 1245; and the Grand coutumier de Normandie (Great customary of Normandy, originally Summa de legibus Normanniae in curia laïcali), authored between 1235 and 1245.',),
 ('What Norman law wasdeveloped between 1000 and 1300?',),
 [('',)],
 [tensor([-1])]]

## Required preprocessing

Neural networks cannot direcly deal with strings. We have first to preprocess the dataset in three main ways:
 1. Tokenise the strings into its words
 2. Build a vocabulary out of these words
 3. Embed each word into a vector, so that each sentence becomes a list of vectors

The first two steps are performed by the `PreprocessTextQA`. The embedding will be added directly to the model.

In [6]:
from gdeep.data import PreprocessTextQA

prec = PreprocessTextQA((dl_tr_str, dl_ts_str))

(dl_tr, dl_ts) = prec.build_dataloaders(batch_size=3)


In [7]:
aa = next(iter(dl_tr))
aa

[tensor([[[1133, 1118, 1031,  ..., 2567, 2567, 2567],
          [  56,  158, 1133,  ..., 2567, 2567, 2567]],
 
         [[1133, 1118, 1031,  ..., 2567, 2567, 2567],
          [  49,   46,  106,  ..., 2567, 2567, 2567]],
 
         [[ 943,  384,   55,  ..., 2567, 2567, 2567],
          [  49,   46,    1,  ..., 2567, 2567, 2567]]]),
 tensor([[51, 55],
         [30, 33],
         [21, 24]])]

## Define and train your model

The model for QA shall accept as input the context and the question and return the probabilities for the initial and final token of the answer in the input context. The output than, is a pair of logits.

In [8]:
from torch.nn import Transformer
from torch.optim import Adam, SparseAdam, SGD
import copy
# my simple transformer model
class QATransformer(nn.Module):

    def __init__(self, src_vocab_size, tgt_vocab_size, embed_dim):
        super(QATransformer, self).__init__()
        self.transformer = Transformer(d_model=embed_dim,
                                       nhead=2,
                                       num_encoder_layers=1,
                                       num_decoder_layers=1,
                                       dim_feedforward=512,
                                       dropout=0.1)
        self.embedding_src = nn.Embedding(src_vocab_size, embed_dim, sparse=True)
        self.embedding_tgt = nn.Embedding(tgt_vocab_size, embed_dim, sparse=True)
        self.generator = nn.Linear(embed_dim, 2)
        
    def forward(self, X):
        #print(X.shape)
        src = X[:,0,:]
        tgt = X[:,1,:]
        #print(src.shape, tgt.shape)
        src_emb = self.embedding_src(src)
        tgt_emb = self.embedding_tgt(tgt)
        #print(src_emb.shape, tgt_emb.shape)
        self.outs = self.transformer(src_emb, tgt_emb)
        #print(outs.shape)
        logits = self.generator(self.outs)
        #print(logits.shape)
        #out = torch.topk(logits, k=1, dim=2).indices.reshape(-1,44)
        #print(out, out.shape)
        return logits
    
    def __deepcopy__(self, memo):
        """this is needed to make sure that the 
        non-leaf nodes do not
        interfere with copy.deepcopy()
        """
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        for k, v in self.__dict__.items():
            setattr(result, k, copy.deepcopy(v, memo))
        return result
    
    def encode(self, src, src_mask):
        """this method is used only at the inference step"""
        return self.transformer.encoder(
                            self.embedding_src(src), src_mask)

    def decode(self, tgt, memory, tgt_mask):
        """this method is used only at the inference step"""
        return self.transformer.decoder(
                          self.embedding_tgt(tgt), memory,
                          tgt_mask)

In [9]:
vocab_size = 5500

src_vocab_size = vocab_size
tgt_vocab_size = vocab_size
emb_size = 64

model = QATransformer(src_vocab_size, tgt_vocab_size, emb_size)
print(model)

QATransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
          )
          (linear1): Linear(in_features=64, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=64, bias=True)
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0): TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
            (out_pr

## Define the loss function

This loss function is a adapted version of the Cross Entropy for the trnasformer architecture.

In [10]:

def loss_fn(output_of_network, label_of_dataloader):
    #print(output_of_network.shape, label_of_dataloader.shape)
    tgt_out = label_of_dataloader
    #print(tgt_out)
    logits = output_of_network
    cel = nn.CrossEntropyLoss()
    return cel(logits, tgt_out)


In [11]:
# prepare a pipeline class with the model, dataloaders loss_fn and tensorboard writer
pipe = Pipeline(model, (dl_tr, dl_ts), loss_fn, writer)

# train the model
pipe.train(SGD, 3, False, {"lr":0.01}, {"batch_size":16})

Epoch 1
-------------------------------
Epoch training loss: 5.617112 	Epoch training accuracy: 1.56%                                                            
Time taken for this epoch: 4.00s
Learning rate value: 0.01000000


/Users/matteocaorsi/Desktop/giotto-deep/gdeep/pipeline/pipeline.py:315: UserWarning:

Cannot store data in the PR curve



Validation results: 
 Accuracy: 0.00%,                 Avg loss: 5.531766 

Epoch 2
-------------------------------
Epoch training loss: 5.451338 	Epoch training accuracy: 1.56%                                                            
Time taken for this epoch: 4.00s
Learning rate value: 0.01000000
Validation results: 
 Accuracy: 0.00%,                 Avg loss: 5.473946 

Epoch 3
-------------------------------
Epoch training loss: 5.414203 	Epoch training accuracy: 2.73%                                                           
Time taken for this epoch: 4.00s
Learning rate value: 0.01000000
Validation results: 
 Accuracy: 1.56%,                 Avg loss: 5.367303 



(5.367303133010864, 1.5625)

## Answering questions!

Here we have a question and its associated context:

In [12]:
bb = next(iter(dl_ts_str))
bb[:2]

[('Computational complexity theory is a branch of the theory of computation in theoretical computer science that focuses on classifying computational problems according to their inherent difficulty, and relating those classes to each other. A computational problem is understood to be a task that is in principle amenable to being solved by a computer, which is equivalent to stating that the problem may be solved by mechanical application of mathematical steps, such as an algorithm.',),
 ('What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?',)]


Get the vocabulary and numericize the question and context to then input both to the model.

In [13]:
voc = prec.vocabulary
context = prec.tokenizer(bb[0][0])
question = prec.tokenizer(bb[1][0])

# get the indexes in the vocabulary of the tokens
context_idx = torch.tensor(list(map(voc.__getitem__,context)))
question_idx = torch.tensor(list(map(voc.__getitem__,question)))

In [14]:
length_to_pad = aa[0].shape[-1]
pad_fn = lambda item : torch.cat([item, prec.pad_item * torch.ones(length_to_pad - item.shape[0])])

# these tansors are ready for be fitted into teh model
context_ready_for_model = pad_fn(context_idx)
question_ready_for_model = pad_fn(question_idx)

Put the two tensors of context and question together and input them to the model

In [15]:
inp = torch.stack((context_ready_for_model, question_ready_for_model)).reshape(1,*aa[0].shape[1:]).long()
out = model(inp)

The output is the ligits for the start and end tokens of the answer. It is now time to extract them with `torch.argmax`

In [22]:
answer_idx = torch.argmax(out, dim=1)

try:
    print("The model proposes: '", context[answer_idx[0][0]],"...'")
except IndexError:
    print("The model was not able to find the answer.")
print("The actual answer was: '" + bb[2][0][0]+"'")

The model proposes: ' computational ...'
The actual answer was: 'Computational complexity theory'


# Extract inner data from your models

In [17]:
from gdeep.models import ModelExtractor

me = ModelExtractor(pipe.model, loss_fn)

lista = me.get_layers_param()

for k, item in lista.items():
    print(k,item.shape)


transformer.encoder.layers.0.self_attn.in_proj_weight torch.Size([192, 64])
transformer.encoder.layers.0.self_attn.in_proj_bias torch.Size([192])
transformer.encoder.layers.0.self_attn.out_proj.weight torch.Size([64, 64])
transformer.encoder.layers.0.self_attn.out_proj.bias torch.Size([64])
transformer.encoder.layers.0.linear1.weight torch.Size([512, 64])
transformer.encoder.layers.0.linear1.bias torch.Size([512])
transformer.encoder.layers.0.linear2.weight torch.Size([64, 512])
transformer.encoder.layers.0.linear2.bias torch.Size([64])
transformer.encoder.layers.0.norm1.weight torch.Size([64])
transformer.encoder.layers.0.norm1.bias torch.Size([64])
transformer.encoder.layers.0.norm2.weight torch.Size([64])
transformer.encoder.layers.0.norm2.bias torch.Size([64])
transformer.encoder.norm.weight torch.Size([64])
transformer.encoder.norm.bias torch.Size([64])
transformer.decoder.layers.0.self_attn.in_proj_weight torch.Size([192, 64])
transformer.decoder.layers.0.self_attn.in_proj_bias t

In [18]:
DEVICE = torch.device("cpu")
x = next(iter(dl_tr))[0]
pipe.model.eval()
pipe.model(x.to(DEVICE))

list_activations = me.get_activations(x)
len(list_activations)


30

In [19]:
x = next(iter(dl_tr))[0][0]
if x.dtype is not torch.int64:
    res = me.get_decision_boundary(x, n_epochs=1)
    res.shape

In [20]:
x, target = next(iter(dl_tr))
if x.dtype is torch.float:
    for gradient in me.get_gradients(x, target=target)[1]:
        print(gradient.shape)

# Visualise activations and other topological aspects of your model

In [21]:
from gdeep.visualisation import Visualiser

vs = Visualiser(pipe)

vs.plot_data_model()
#vs.plot_activations(x)
#vs.plot_persistence_diagrams(x)
